# Kivy Runner

Es una aplicaicón diseñada para ejecutar nuestros scripts de python sin tener que generar el APK (paquete para Android).

Debemos copiar nuestro script a la carpeta `/storage/sdcard0/kivy` 

De no ubicarse en este directorio, utilzando ADB es posible ubicarlo con `adb shell` con el comando `busybox find / -iname "kivy" 2>/dev/null`.


Para que nuestra aplicación pueda ejecutarse, debemos utilzar agregar el siguiente archivo para que Kivy Runner pueda utilizarlo:

### `android.txt`


In [3]:
author = "PyCon Crew"
version = 1.0
orientation = 'vertical'

# Ejercicio

Utilizar la máquina virtual o nuestra propia máquina para mover un main.py, un android.txt en una carpeta de Kivy Runner. 

# Compartiendo Carpetas 

![](images/shared.png)



![](images/usb.png)

# Buildozer

Buildozer es una [herramienta del proyecto Kivy](https://github.com/kivy/buildozer) para crear paquetes para Android y iOS facilmente.

Utiliza un archivo `buildozer.spec` dónde se definen algunos prámetros de configuración como:
* requerimentos 
* titulo
* icon
* modulos incluidos (de PyPi y de Garden)

Buildozer crea estos archivos por nosotros.

# Instalación

```python
sudo pip install -U buildozer
```

# Creación de `buildozer.spec`
```bash
buildozer init 
```

# Configuración de `buildozer.spec`
## Algunas configuracione a tener en cuenta
```ini
# (list) Permissions
android.permissions = INTERNET,ACCESS_FINE_LOCATION,ACCESS_COARSE_LOCATION

# (int) Android API to use
#android.api = 14

# (int) Minimum API required (8 = Android 2.2 devices)
#android.minapi = 8

# (int) Android SDK version to use
#android.sdk = 21

# (str) Android NDK version to use
#android.ndk = 9

# (bool) Use --private data storage (True) or --dir public storage (False)
#android.private_storage = True

# (str) Android NDK directory (if empty, it will be automatically downloaded.)
#android.ndk_path =

# (str) Android SDK directory (if empty, it will be automatically downloaded.)
android.sdk_path = 

# (str) Android entry point, default is ok for Kivy-based app
#android.entrypoint = org.renpy.android.PythonActivity

# (list) List of Java .jar files to add to the libs so that pyjnius can access
# their classes. Don't add jars that you do not need, since extra jars can slow
# down the build process. Allows wildcards matching, for example:
# OUYA-ODK/libs/*.jar
#android.add_jars = foo.jar,bar.jar,path/to/more/*.jar

# (list) List of Java files to add to the android project (can be java or a
# directory containing the files)
#android.add_src =

# (str) XML file to include as an intent filters in <activity> tag
#android.manifest.intent_filters =

# (list) Android additionnal libraries to copy into libs/armeabi
#android.add_libs_armeabi = libs/android/*.so

# (bool) Indicate whether the screen should stay on
# Don't forget to add the WAKE_LOCK permission if you set this to True
#android.wakelock = False

# (list) Android application meta-data to set (key=value format)
#android.meta_data =

# (list) Android library project to add (will be added in the
# project.properties automatically.)
#android.library_references =
```

# Compilación de la Aplicación para Android
```bash
$ buildozer android_new debug
# Ensure build layout
# Check configuration tokens
# Preparing build
# Check requirements for android
# Install platform
# Apache ANT found at /Users/nahuel/.buildozer/android/platform/apache-ant-1.9.4
# Android SDK found at /usr/local/Cellar/android-sdk/24.4.1_1
# Android NDK found at /Users/nahuel/.buildozer/android/platform/android-ndk-r9c
# Check application requirements
# Check garden requirements
# Compile platform
# Build the application #2
# Package the application
# APK MyApplication-0.1-debug.apk available in the bin directory
```

Esto puede tartar un tiempo la primera vez y la carpeta oculta de Buildozer, puede llegar a ocuar **1.1GB**.

![](images/meme_kivy.jpg)

# Android Debug Bridge

`adb` es una utilidad de línea de comandos que sirve para:
* Ver los mensajes (logs) de Android (`adb logcat`)
* Crear una shell en el 

* adb push <local>... <remote> Enviar un archivo/directorio al dispositivo
* adb pull <remote>... <local> Traer ar
                               - copy files/dirs from device
                                 (-a preserves file timestamp and mode)
  adb sync [ <directory> ]     - copy host->device only if changed
                                 (-l means list but don't copy)
  adb shell [-e escape] [-n] [-Tt] [-x] [command]
                               - run remote shell command (interactive shell if no command given)
                                 (-e: choose escape character, or "none"; default '~')
                                 (-n: don't read from stdin)
                                 (-T: disable PTY allocation)
                                 (-t: force PTY allocation)
                                 (-x: disable remote exit codes and stdout/stderr separation)
  adb emu <command>            - run emulator console command
  adb logcat [ <filter-spec> ] - View device log
  adb forward --list           - list all forward socket connections.
                                 the format is a list of lines with the following format:
                                    <serial> " " <local> " " <remote> "\n"

# Utilización

```bash
adb logcat | grep python
```

# Recomendaciones

Buildozer facilita enormemente la tarea de crear aplicaciones para android, descargando los SDKs y compilando, pero genera una cantidad importante de archivos temporales, que si bien aceleran la compilación, pueden ocupar bastante si se tienen múltiples proyectos. Un paleativo puede ser quitar la carpeta **`.buildozer`** en los proyectos que no estemos trabajando en el momento.

# APIs Nativas de Android

## Plyer

Plyer ofrece de forma Pythónica [muchas APIS](https://github.com/kivy/plyer/tree/master/examples) que se pueden usar sin necesidad de Java:
```python
from kivy.lang import Builder
from plyer import gps
from kivy.app import App
from kivy.properties import StringProperty
from kivy.clock import Clock, mainthread

kv = '''
BoxLayout:
    orientation: 'vertical'

    Label:
        text: app.gps_location

    Label:
        text: app.gps_status

    BoxLayout:
        size_hint_y: None
        height: '48dp'
        padding: '4dp'

        ToggleButton:
            text: 'Start' if self.state == 'normal' else 'Stop'
            on_state:
                app.start(1000, 0) if self.state == 'down' else \
                app.stop()
'''


class GpsTest(App):

    gps_location = StringProperty()
    gps_status = StringProperty('Click Start to get GPS location updates')

    def build(self):
        try:
            gps.configure(on_location=self.on_location,
                          on_status=self.on_status)
        except NotImplementedError:
            import traceback
            traceback.print_exc()
            self.gps_status = 'GPS is not implemented for your platform'

        return Builder.load_string(kv)

    def start(self, minTime, minDistance):
        gps.start(minTime, minDistance)

    def stop(self):
        gps.stop()

    @mainthread
    def on_location(self, **kwargs):
        self.gps_location = '\n'.join([
            '{}={}'.format(k, v) for k, v in kwargs.items()])

    @mainthread
    def on_status(self, stype, status):
        self.gps_status = 'type={}\n{}'.format(stype, status)

    def on_pause(self):
        gps.stop()
        return True

    def on_resume(self):
        gps.start(1000, 0)
        pass

if __name__ == '__main__':
    GpsTest().run()
```

# on_pause, on_resume

Esots métodos definen como almacenar el estado de la aplicación. Cada vez que nuestra aplicación pierde foco, se destruye y es necesario restablecerla en el estado anterior.

## Ejercicio
Agregar a nuestra aplicación los métodos on_pause y on_resume, persistir el estado en un archivo utiliando `kivy.storage.jsonstore.JsonStore`
